### from : 01.indexing.md

In [1]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_REGION = os.getenv("PINECONE_INDEX_REGION")
PINECONE_INDEX_CLOUD = os.getenv("PINECONE_INDEX_CLOUD")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_INDEX_METRIC = os.getenv("PINECONE_INDEX_METRIC")
PINECONE_INDEX_DIMENSION = int(os.getenv("PINECONE_INDEX_DIMENSION"))

print("환경 변수 로딩 완료")

환경 변수 로딩 완료


In [2]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=PINECONE_API_KEY)

# 인덱스 생성 여부 확인 및 생성
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=PINECONE_INDEX_DIMENSION,
        metric=PINECONE_INDEX_METRIC,
        spec=ServerlessSpec(
            region=PINECONE_INDEX_REGION,
            cloud=PINECONE_INDEX_CLOUD
        )
    )
    print(f"인덱스 '{PINECONE_INDEX_NAME}' 생성 완료")
else:
    print(f"인덱스 '{PINECONE_INDEX_NAME}'가 이미 존재합니다.")

# 인덱스 객체 가져오기
index = pc.Index(PINECONE_INDEX_NAME)
print("Pinecone 인덱스 연결 완료")


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


인덱스 'ir-embeddings' 생성 완료
Pinecone 인덱스 연결 완료


In [5]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(
    model=OPENAI_EMBEDDING_MODEL,
    openai_api_key=OPENAI_API_KEY
)

print("임베딩 모델 로딩 완료")

임베딩 모델 로딩 완료


In [7]:
import pandas as pd

# 문서 데이터 로드 (documents.csv 파일 경로를 지정하세요)
documents_df = pd.read_csv("../../datas/documents.csv")

# 인덱싱용 리스트 생성: (id, 텍스트)
docs_to_index = []
for idx, row in documents_df.iterrows():
    doc_id = row['doc_id']
    content = row['content']
    docs_to_index.append((doc_id, content))

print(f"총 {len(docs_to_index)}개의 문서를 인덱싱 준비")

총 30개의 문서를 인덱싱 준비


In [8]:
from langchain_pinecone import PineconeVectorStore

# Pinecone 벡터 스토어 연결
vector_store = PineconeVectorStore(
    index_name=PINECONE_INDEX_NAME,
    embedding=embedding_model
)

print("벡터 스토어 연결 완료")

벡터 스토어 연결 완료


In [9]:
# Pinecone에 문서 업서트
from langchain.schema import Document

# 문서를 벡터로 변환하여 업서트
for doc_id, content in docs_to_index:
    # LangChain Document 생성
    doc = Document(page_content=content, metadata={"doc_id": doc_id})
    
    # 벡터 저장
    vector_store.add_documents([doc])

print("모든 문서 업서트 완료")

모든 문서 업서트 완료


In [10]:
# 테스트 검색 예시
query = "제주도 관광 명소"
results = vector_store.similarity_search(query, k=5)
print("질의:", query)
for rank, doc in enumerate(results, start=1):
    print(f"{rank}. 문서 ID: {doc.metadata['doc_id']}")

질의: 제주도 관광 명소
1. 문서 ID: D1
2. 문서 ID: D12
3. 문서 ID: D2
4. 문서 ID: D13
5. 문서 ID: D8
